In [1]:
# DPLL

Code hieronder is puur om de literal te kiezen die het meeste voorkomt 

In [ ]:
from collections import defaultdict 

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals


# Mijn versie van DPLL
Mocht je willen kijken hoe mijn versie van DPLL eruit ziet


In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path


    line = file.readline()

    components = line.strip().split(" ")

    if len(components)!= 4 or components[0]!="p" or components[1]!="cnf":
      print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES")
      exit(1)

    num_vars=int(components[2])
    num_clauses=int(components[3])

    clauses=[]

    line=file.readline()
    while(line):
       numbers = [int(x) for x in line.strip().split(" ")]

       if(numbers[-1]!=0):
          print("Wrong format! Clause lines must be terminated with a 0")

       clauses.append(numbers[:-1])

       line=file.readline()


    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snelle simplificatie:
      - verwijder tautologieën
      - unit propagation (herhaald, met queue)
      - pure literal elimination (globaal per iteratie)
    Geeft (nieuwe_clauses, truth_list) terug.
    """

    # verzameling van alle geforceerde waar-literals
    truth_set = set()

    # we werken op een lokale kop zodat het origineel niet stuk gaat
    clauses = [list(cl) for cl in clauses]

    changed = True
    while changed:
        changed = False

        # -------------------------
        # 1. Tautologieën + directe unit clauses
        # -------------------------
        unit_queue = deque()
        new_clauses: List[List[int]] = []

        for clause in clauses:
            # duplicates binnen clausule verwijderen
            lits = set(clause)

            # tautologie? (x en -x in dezelfde clausule)
            if any(-lit in lits for lit in lits):
                # sla deze clausule over
                changed = True
                continue

            # unit clausule?
            if len(lits) == 1:
                lit = next(iter(lits))
                unit_queue.append(lit)
                changed = True
                continue

            new_clauses.append(list(lits))

        clauses = new_clauses

        # -------------------------
        # 2. Unit propagation (met queue)
        # -------------------------
        while unit_queue and clauses:
            u = unit_queue.popleft()

            # conflict: zowel u als -u?
            if -u in truth_set:
                # formule is inconsistent -> leeg clause toevoegen
                return ([[]], list(truth_set))

            if u in truth_set:
                continue  # al verwerkt

            truth_set.add(u)
            changed = True

            new_clauses = []
            for clause in clauses:
                # clause is voldaan?
                if u in clause:
                    continue

                # tegensgestelde literal verwijderen
                if -u in clause:
                    new_clause = [lit for lit in clause if lit != -u]
                    if len(new_clause) == 0:
                        # lege clausule -> UNSAT signaal voor DPLL
                        new_clauses.append(new_clause)
                    elif len(new_clause) == 1:
                        # nieuwe unit literal gevonden
                        unit_queue.append(new_clause[0])
                        new_clauses.append(new_clause)
                    else:
                        new_clauses.append(new_clause)
                else:
                    new_clauses.append(clause)

            clauses = new_clauses

        # Als we al een lege clausule hebben, hoeven we niet verder
        if any(len(cl) == 0 for cl in clauses):
            break

        # -------------------------
        # 3. Pure literal elimination
        # -------------------------
        # tel voorkomen van iedere literal
        lit_count = defaultdict(int)
        for clause in clauses:
            for lit in clause:
                lit_count[lit] += 1

        # pure literals = lit met count>0 en count(-lit)==0
        pure_lits = [
            lit for lit, cnt in lit_count.items()
            if cnt > 0 and lit_count.get(-lit, 0) == 0
        ]

        if pure_lits:
            changed = True
            pure_set = set(pure_lits)
            # voeg toe aan assignment
            for p in pure_set:
                truth_set.add(p)

            # alle clausules die een pure literal bevatten zijn voldaan -> weg
            new_clauses = []
            for clause in clauses:
                if any(lit in pure_set for lit in clause):
                    continue
                new_clauses.append(clause)
            clauses = new_clauses

    # einde while: geen veranderingen meer
    return clauses, list(truth_set)


##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0


def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:
    
    # Check how many times DPLL is called
    global DPLL_CALLS
    DPLL_CALLS += 1
    print("DPLL call number:", DPLL_CALLS)
    if assignment is None:
        assignment = []

    # Simplify the clauses
    simplified_clauses, implied_truths = simplify(clauses)

    # Add implied truths to assignment
    full_assignment = assignment.copy()
    for literal in implied_truths:
        if -literal in full_assignment:
            # conflict in assignment
            return False, []
        if literal not in full_assignment:
            full_assignment.append(literal)

    # Check for empty clause set -> return UNSAT
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # Check if all clauses are satisfied
    if not simplified_clauses:
        return True, full_assignment

    # DLIS step -> choose literal to split on (assign true or false)        
    split_lit = choose_literal_dlis(simplified_clauses)

    sat, model = dpll(
    simplified_clauses + [[split_lit]],
    num_vars,
    full_assignment + [split_lit])

    if sat:
        return True, model

    sat,model = dpll(
    simplified_clauses + [[-split_lit]],
    num_vars,
    full_assignment + [-split_lit])

    return sat, model 


if __name__ == "__main__":
    # clauses, num_vars = parse_dimacs("inst16.cnf")
    # choose_literal_dlis(clauses)
    # sat, assignment = dpll(clauses, num_vars)
    # if sat:
    #     print("SAT")
    #     print(assignment)   
    # else:
    #     print("UNSAT")
    clauses, num_vars = parse_dimacs("inst16.cnf")

    import time
    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")  

1 2 3 4 5 6 7 8 9 0


# Fast Solver
versie waarbij simplify verkort is en JW heuristic

In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

##################################################################################
# DIMACS I/O (zoals je al had)
##################################################################################

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path

    # lees header
    line = file.readline()
    while line and line.startswith("c"):
        # commentregels overslaan
        line = file.readline()

    components = line.strip().split()

    if len(components) != 4 or components[0] != "p" or components[1] != "cnf":
        print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES'")
        exit(1)

    num_vars = int(components[2])
    num_clauses = int(components[3])

    clauses = []

    line = file.readline()
    while line:
        # sla commentregels over
        if line.startswith("c") or not line.strip():
            line = file.readline()
            continue

        numbers = [int(x) for x in line.strip().split()]
        # DIMACS clausules kunnen over meerdere regels lopen, maar in jullie geval
        # lijkt het per regel één clausule + 0. Dit is de simpele variant:
        if numbers[-1] != 0:
            print("Wrong format! Clause lines must be terminated with a 0")
            exit(1)

        clauses.append(numbers[:-1])
        line = file.readline()

    if close:
        file.close()

    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()

##################################################################################
# Snelle simplify: alleen unit propagation + tautologieën weggooien
##################################################################################

def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snelle simplificatie:
      - verwijder tautologieën
      - verzamel unit clauses
      - unit propagation met een queue

    GEEN pure literal elimination in de recursie (scheelt enorm veel tijd).

    Geeft (nieuwe_clauses, implied_literals) terug.
    """

    truth_set = set()
    unit_queue = deque()

    new_clauses: List[List[int]] = []

    # Eerste pass: duplicates eruit, tautologieën weg, units verzamelen
    for clause in clauses:
        if not clause:
            # lege clause => direct conflict
            return [[]], list(truth_set)

        seen = set()
        lits = []
        taut = False

        for lit in clause:
            if -lit in seen:
                # tautologie (x en -x in dezelfde clause) => altijd waar, skip
                taut = True
                break
            if lit not in seen:
                seen.add(lit)
                lits.append(lit)

        if taut:
            continue  # clause is altijd waar, hoeven we niet mee te nemen

        if len(lits) == 0:
            # lege clause => conflict
            return [[]], list(truth_set)
        elif len(lits) == 1:
            # unit clause
            unit_queue.append(lits[0])
        else:
            new_clauses.append(lits)

    clauses = new_clauses

    # Unit propagation
    while unit_queue and clauses:
        u = unit_queue.popleft()

        # conflict met eerdere literal?
        if -u in truth_set:
            return [[]], list(truth_set)

        if u in truth_set:
            continue  # al verwerkt

        truth_set.add(u)

        new_clauses = []
        for clause in clauses:
            # clause is voldaan?
            if u in clause:
                continue

            # verwijder tegensgestelde literal
            if -u in clause:
                reduced = [lit for lit in clause if lit != -u]
                if len(reduced) == 0:
                    # lege clause => conflict
                    return [[]], list(truth_set)
                if len(reduced) == 1:
                    # nieuwe unit
                    unit_queue.append(reduced[0])
                new_clauses.append(reduced)
            else:
                new_clauses.append(clause)

        clauses = new_clauses

    return clauses, list(truth_set)

##################################################################################
# Heuristiek: Jeroslow–Wang (JW)
##################################################################################

def choose_literal_jw(clauses: List[List[int]]) -> int:
    """
    Jeroslow–Wang heuristiek:

      score(l) = som over clausules C die l bevatten van 2^(-|C|)

    Korte clausules wegen zwaarder; dat helpt vaak sterk bij SAT-instances (zoals Sudoku).
    """
    scores: Dict[int, float] = defaultdict(float)

    for clause in clauses:
        # gewicht van de clause (korte clausules krijgen meer gewicht)
        w = 2.0 ** (-len(clause))
        for lit in clause:
            scores[lit] += w

    if not scores:
        # zou eigenlijk niet mogen gebeuren als we hier komen,
        # maar voor de zekerheid:
        return 0

    return max(scores, key=scores.get)

##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:
    """
    Klassieke DPLL:
      - simplify (unit propagation)
      - check SAT/UNSAT
      - kies literal via JW
      - branch: lit = True, daarna lit = False

    Geen clause learning, geen watched literals (nog), wel veel lichter dan je vorige versie.
    """

    global DPLL_CALLS
    DPLL_CALLS += 1
    if DPLL_CALLS % 100000 == 0:
      print("DPLL call number:", DPLL_CALLS)
      
    if assignment is None:
        assignment = []

    # Simplify met unit propagation
    simplified_clauses, implied_truths = simplify(clauses)

    # Voeg implied literals toe aan assignment
    full_assignment = assignment.copy()
    assigned_set = set(full_assignment)

    for lit in implied_truths:
        if -lit in assigned_set:
            # conflict in assignment
            return False, []
        if lit not in assigned_set:
            full_assignment.append(lit)
            assigned_set.add(lit)

    # Check op conflict: lege clause
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # Geen clausules meer => alle zijn voldaan
    if not simplified_clauses:
        return True, full_assignment

    # Kies splitsliteral met Jeroslow–Wang
    split_lit = choose_literal_jw(simplified_clauses)
    if split_lit == 0:
        # fallback: als er toch iets misging
        split_lit = simplified_clauses[0][0]

    # Branch 1: split_lit = True
    sat, model = dpll(
        simplified_clauses + [[split_lit]],
        num_vars,
        full_assignment + [split_lit]
    )
    if sat:
        return True, model

    # Branch 2: split_lit = False
    sat, model = dpll(
        simplified_clauses + [[-split_lit]],
        num_vars,
        full_assignment + [-split_lit]
    )
    return sat, model

##################################################################################
# Main (optioneel testje op een CNF-file)
##################################################################################

if __name__ == "__main__":
    # Voorbeeld: rechtstreeks een .cnf bestand draaien
    # Pas dit pad aan naar jouw gegenereerde DIMACS file (bijv. enc16.cnf)
    cnf_path = "enc16.cnf"

    clauses, num_vars = parse_dimacs(cnf_path)

    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")

    if sat:
        # Optioneel: assignment sorteren en afdrukken
        assignment = sorted(assignment, key=lambda x: abs(x))
        print("Model (eerste 50 lits):", assignment[:50])


# DLIS en JW gecombineerde heuristic
Lijkt snel te runnen, sneller dan bovenstaande code


In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path


    line = file.readline()

    components = line.strip().split(" ")

    if len(components)!= 4 or components[0]!="p" or components[1]!="cnf":
      print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES")
      exit(1)

    num_vars=int(components[2])
    num_clauses=int(components[3])

    clauses=[]

    line=file.readline()
    while(line):
       numbers = [int(x) for x in line.strip().split(" ")]

       if(numbers[-1]!=0):
          print("Wrong format! Clause lines must be terminated with a 0")

       clauses.append(numbers[:-1])

       line=file.readline()


    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snellere simplificatie:
      - unit propagation (herhaald, met queue)
    Geen tautologie-check, geen pure literals (voor Sudoku meestal zinloos).

    Returns:
        (nieuwe_clauses, truth_list)
        - nieuwe_clauses: vereenvoudigde formule
        - truth_list: alle literals die geforceerd True zijn
    """

    truth_set = set()
    truth_list: List[int] = []

    # Werk op een kopie zodat het origineel intact blijft
    clauses = [cl[:] for cl in clauses]

    # Start: zoek alle unit clauses
    unit_queue = deque()
    for clause in clauses:
        if len(clause) == 1:
            unit_queue.append(clause[0])

    # Unit propagation loop
    while unit_queue and clauses:
        u = unit_queue.popleft()

        # Als -u al geforceerd is -> conflict
        if -u in truth_set:
            # Representatie van UNSAT: formule met lege clause
            return ([[]], truth_list)

        # Als u al verwerkt is, overslaan
        if u in truth_set:
            continue

        truth_set.add(u)
        truth_list.append(u)

        new_clauses: List[List[int]] = []
        for clause in clauses:
            # Clause is voldaan als u erin zit
            if u in clause:
                continue

            # Als -u erin zit, verwijder die literal
            if -u in clause:
                new_clause = [lit for lit in clause if lit != -u]

                if len(new_clause) == 0:
                    # lege clause -> UNSAT
                    return ([[]], truth_list)

                if len(new_clause) == 1:
                    # nieuwe unit literal
                    unit_queue.append(new_clause[0])

                new_clauses.append(new_clause)
            else:
                new_clauses.append(clause)

        clauses = new_clauses

    # Geen nieuwe units meer
    return clauses, truth_list


##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1

    # DLIS: kies literal (met teken) met hoogste count
    most_literal = max(counts, key=counts.get)
    return most_literal

def choose_literal_dlis_jw(clauses: List[List[int]]) -> int:
    dlis_counts = defaultdict(int)
    jw_score = defaultdict(float)

    for clause in clauses:
        weight = 2.0 ** (-len(clause))   # Jeroslow–Wang factor
        for lit in clause:
            dlis_counts[lit] += 1
            jw_score[lit] += weight

    # Combineer beide heuristieken
    # α = DLIS gewicht, β = JW gewicht
    α = 1.0
    β = 2.0   # JW iets sterker laten wegen werkt goed voor sudoku

    combined = {
        lit: α * dlis_counts[lit] + β * jw_score[lit]
        for lit in dlis_counts.keys()
    }

    return max(combined, key=combined.get)


def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:

    global DPLL_CALLS
    DPLL_CALLS += 1
    if DPLL_CALLS % 100000 == 0:
        print("DPLL call number:", DPLL_CALLS)

    if assignment is None:
        assignment = []

    # 1. Simplify
    simplified_clauses, implied_truths = simplify(clauses)

    # 2. Implied truths toevoegen aan assignment
    full_assignment = assignment.copy()
    for literal in implied_truths:
        if -literal in full_assignment:
            # conflict in assignment
            return False, []
        if literal not in full_assignment:
            full_assignment.append(literal)

    # 3. UNSAT: lege clause aanwezig?
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # 4. SAT: geen clausules meer
    if not simplified_clauses:
        return True, full_assignment

    # 5. DLIS: kies literal (met teken)
    split_lit = choose_literal_dlis_jw(simplified_clauses)

    # 6. Eerste tak: literal = True
    sat, model = dpll(
        simplified_clauses + [[split_lit]],
        num_vars,
        full_assignment + [split_lit]
    )
    if sat:
        return True, model

    # 7. Tweede tak: literal = False
    sat, model = dpll(
        simplified_clauses + [[-split_lit]],
        num_vars,
        full_assignment + [-split_lit]
    )

    return sat, model


if _name_ == "_main_":
    # clauses, num_vars = parse_dimacs("inst16.cnf")
    # choose_literal_dlis(clauses)
    # sat, assignment = dpll(clauses, num_vars)
    # if sat:
    #     print("SAT")
    #     print(assignment)   
    # else:
    #     print("UNSAT")
    clauses, num_vars = parse_dimacs("inst16.cnf")

    import time
    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")